## Imporing Libraries

In [147]:
!pip install transformers torch Pillow scikit-learn numpy git+https://github.com/openai/CLIP.git ftfy torchvision

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/CLIP.git to /private/var/folders/8f/ydfzy_2j73n93khd4xqvwmzh0000gn/T/pip-req-build-ciame4ru
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/8f/ydfzy_2j73n93khd4xqvwmzh0000gn/T/pip-req-build-ciame4ru
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [97]:
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os
import clip

In [100]:
dataset_path = '/Users/kabir/FRE-7773-Project/data'

In [101]:
def get_image_paths(base_path):
    image_paths = []
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):  
                image_paths.append(os.path.join(root, file))
    return image_paths

# Converting Pickel Files To Usable DataFrames

In [ ]:
import pickle
import os


text_data_path = '/Users/kabir/FRE-7773-Project/data/text_data'  


pickle_files = [
    'categories_dict.p',
    'categories_images_dict.p',
    'img_to_desc.p',
    'item_to_room.p',
    'products_dict.p',
    'room_to_items.p'
]

# Initializing a dictionary to store the data
data_dict = {}

# Loopin g through each pickle file and load the data
for file_name in pickle_files:
    file_path = os.path.join(text_data_path, file_name)
    with open(file_path, 'rb') as file:
        data_dict[file_name] = pickle.load(file)

# Now `data_dict` will have the contents of each pickle file
# For example, to access the image to description mapping:
categories_dict=data_dict['categories_dict.p']
categories_images_dict=data_dict['categories_images_dict.p']
item_to_room=data_dict['item_to_room.p']
products_dict=data_dict['products_dict.p']
room_to_items=data_dict['room_to_items.p']
img_to_desc = data_dict['img_to_desc.p']

In [166]:
# Making a function to preview dictionaries like .head() for dataframes
from itertools import islice
def see_dict(dict_sample):
    first_five_pairs = dict(islice(dict_sample.items(), 5))
    for key, value in first_five_pairs.items():
        print(f'{key}: {value}')

In [ ]:
see_dict(categories_dict)
print("\n")
see_dict(categories_images_dict)
print("\n")
see_dict(item_to_room)
print("\n")
see_dict(products_dict)
print("\n")
see_dict(room_to_items)
print("\n")
see_dict(img_to_desc)

ikea-a-bed-that-folds-away-to-be-a-sofa-by-day__1364309472525-s4.jpg: Bedroom
ikea-a-bright-kitchen-idea-with-custom-character__1364316015597-s4.jpg: Kitchen
ikea-a-clean-way-to-display-or-tidy-away.__1364309466684-s4.jpg: Living Room
ikea-a-colourful-room-for-kids-to-get-creative__1364308442737-s4.jpg: Baby Children Room
ikea-a-contemporary-kitchen-for-big-chefs-and-little-helpers__1364316013714-s4.jpg: Kitchen


500.210.76: Easel
802.538.09: Reversible bed
802.962.48: Quilt cover and pillowcase
502.962.97: Bed pocket
203.086.97: Frame


images/500.210.76.jpg: ['images/room_scenes/ikea-a-colourful-room-for-kids-to-get-creative__1364308442737-s4.jpg', 'images/room_scenes/ikea-artist-studio-by-day-cosy-cocoon-by-night__1364319298429-s4.jpg', 'images/room_scenes/ikea-artist-studio-by-day-cosy-cocoon-by-night-1364319298711.jpg']
images/bed/802.538.09.jpg: ['images/room_scenes/ikea-a-colourful-room-for-kids-to-get-creative__1364308442737-s4.jpg', 'images/room_scenes/ikea-big-world-small-sc

In [159]:
import pandas as pd

# Assuming data_dict is a dictionary containing all loaded pickle data
# Convert each dictionary into a DataFrame
categories_df = pd.DataFrame(list(categories_dict.items()), columns=['Description', 'Room'])
categories_images_df = pd.DataFrame(list(categories_images_dict.items()), columns=['Category', 'Image'])
item_to_room_df = pd.DataFrame(list(item_to_room.items()), columns=['ItemID', 'Room'])
products_df = pd.DataFrame(list(products_dict.items()), columns=['ProductID', 'ProductDetails'])
room_to_items_df = pd.DataFrame(list(room_to_items.items()), columns=['RoomImage', 'Items'])
img_to_desc_df = pd.DataFrame(list(img_to_desc.items()), columns=['Image', 'Description'])


# Room Scenes and Products Data EDA and Extraction

In [116]:
categories_df.head()


,Description,Room
0,ikea-a-bed-that-folds-away-to-be-a-sofa-by-day...,Bedroom
1,ikea-a-bright-kitchen-idea-with-custom-charact...,Kitchen
2,ikea-a-clean-way-to-display-or-tidy-away.__136...,Living Room
3,ikea-a-colourful-room-for-kids-to-get-creative...,Baby Children Room
4,ikea-a-contemporary-kitchen-for-big-chefs-and-...,Kitchen


In [117]:
categories_images_df.head()


,Category,Image
0,500.210.76,Easel
1,802.538.09,Reversible bed
2,802.962.48,Quilt cover and pillowcase
3,502.962.97,Bed pocket
4,203.086.97,Frame


In [160]:
item_to_room_df.to_csv('/Users/kabir/FRE-7773-Project/data/item_to_room.csv')

In [161]:
def transform_dataframe(file_path):
    # Loading the CSV file
    df = pd.read_csv(file_path)

    # Preparing a list for room-item pairs
    room_item_pairs = []

    # Processing each row to extract room-item pairs
    for _, row in df.iterrows():
        item = row['ItemID']
        # Evaluating the string representation of a list to actual list
        rooms = eval(row['Room'])

        for room in rooms:
            room_item_pairs.append([room, item])

    # Converting the list of pairs to a DataFrame
    transformed_df = pd.DataFrame(room_item_pairs, columns=['Room', 'Item'])

    return transformed_df

file_path = '/Users/kabir/FRE-7773-Project/data/item_to_room.csv'
transformed_df = transform_dataframe(file_path)

# Display the first few rows of the transformed DataFrame
print(transformed_df.head())

                                                Room  \
0  images/room_scenes/ikea-a-colourful-room-for-k...   
1  images/room_scenes/ikea-artist-studio-by-day-c...   
2  images/room_scenes/ikea-artist-studio-by-day-c...   
3  images/room_scenes/ikea-a-colourful-room-for-k...   
4  images/room_scenes/ikea-big-world-small-scale_...   

                        Item  
0      images/500.210.76.jpg  
1      images/500.210.76.jpg  
2      images/500.210.76.jpg  
3  images/bed/802.538.09.jpg  
4  images/bed/802.538.09.jpg  


In [162]:
transformed_df.head()

,Room,Item
0,images/room_scenes/ikea-a-colourful-room-for-k...,images/500.210.76.jpg
1,images/room_scenes/ikea-artist-studio-by-day-c...,images/500.210.76.jpg
2,images/room_scenes/ikea-artist-studio-by-day-c...,images/500.210.76.jpg
3,images/room_scenes/ikea-a-colourful-room-for-k...,images/bed/802.538.09.jpg
4,images/room_scenes/ikea-big-world-small-scale_...,images/bed/802.538.09.jpg


In [163]:
def extract_product_id(item_path):
   return item_path.replace('.jpg', '').split('/')[-1]
transformed_df['Product ID'] = transformed_df['Item'].apply(extract_product_id)

# Display the DataFrame to verify the results
transformed_df.head()


                                                Room  \
0  images/room_scenes/ikea-a-colourful-room-for-k...   
1  images/room_scenes/ikea-artist-studio-by-day-c...   
2  images/room_scenes/ikea-artist-studio-by-day-c...   
3  images/room_scenes/ikea-a-colourful-room-for-k...   
4  images/room_scenes/ikea-big-world-small-scale_...   

                        Item  Product ID  
0      images/500.210.76.jpg  500.210.76  
1      images/500.210.76.jpg  500.210.76  
2      images/500.210.76.jpg  500.210.76  
3  images/bed/802.538.09.jpg  802.538.09  
4  images/bed/802.538.09.jpg  802.538.09  


In [164]:
transformed_df.head()

,Room,Item,Product ID
0,images/room_scenes/ikea-a-colourful-room-for-k...,images/500.210.76.jpg,500.210.76
1,images/room_scenes/ikea-artist-studio-by-day-c...,images/500.210.76.jpg,500.210.76
2,images/room_scenes/ikea-artist-studio-by-day-c...,images/500.210.76.jpg,500.210.76
3,images/room_scenes/ikea-a-colourful-room-for-k...,images/bed/802.538.09.jpg,802.538.09
4,images/room_scenes/ikea-big-world-small-scale_...,images/bed/802.538.09.jpg,802.538.09


In [165]:
transformed_df.to_csv('/Users/kabir/FRE-7773-Project/data/clean_data/room_products.csv')

# Products EDA and Extraction


In [119]:
products_df.head()

,ProductID,ProductDetails
0,500.210.76,"{'name': 'MLA', 'type': 'Easel', 'color': 'Sof..."
1,802.538.09,"{'name': 'KURA', 'type': 'Reversible bed', 'co..."
2,802.962.48,"{'name': 'STICKAT', 'type': 'Quilt cover and p..."
3,502.962.97,"{'name': 'STICKAT', 'type': 'Bed pocket', 'col..."
4,203.086.97,"{'name': 'TROFAST', 'type': 'Frame', 'color': ..."


In [120]:
products_df[['Name', 'Type','Color','Size','Image','Description']] = products_df['ProductDetails'].apply(pd.Series)

In [121]:
# Remove the 'ProductDetails' column
products_df = products_df.drop('ProductDetails', axis=1)

# Remove the specific substring "View more product information" from the 'Description' column
products_df['Description'] = products_df['Description'].str.replace('View more product information', '', regex=False).str.strip()

# The DataFrame 'df' now has the 'ProductDetails' column removed, and the specific substring is removed from 'Description'.


In [122]:
products_df.head()

,ProductID,Name,Type,Color,Size,Image,Description
0,500.210.76,MLA,Easel,"Softwood,white",,images/objects/500.210.76.jpg,Your child can use it for different purposes. ...
1,802.538.09,KURA,Reversible bed,"White,pine",90x200 cm,images/objects/802.538.09.jpg,Turned upside down the bed quickly converts fr...
2,802.962.48,STICKAT,Quilt cover and pillowcase,"Turquoise,lilac","150x200,50x80 cm",images/objects/802.962.48.jpg,"Made from 100% cotton, a natural material that..."
3,502.962.97,STICKAT,Bed pocket,Orange,39x30 cm,images/objects/502.962.97.jpg,Clever storage solution that you can hang on o...
4,203.086.97,TROFAST,Frame,Light white stained pine,94x52 cm,images/objects/203.086.97.jpg,A playful and sturdy storage series for storin...


In [123]:
room_to_items_df.head()

,RoomImage,Items
0,images/room_scenes/ikea-a-colourful-room-for-k...,"[easel, reversible bed, pillow, quilt cover an..."
1,images/room_scenes/ikea-a-cosy-base-camp-for-y...,"[wardrobe, ext bed frame with slatted bed base..."
2,images/room_scenes/ikea-a-fresh-look-for-a-fir...,"[cot with drawers, pillow, quilt cover,pillowc..."
3,images/room_scenes/ikea-a-soft-pastel-place-to...,"[cot, pillow, quilt cover,pillowcase for cot, ..."
4,images/room_scenes/ikea-a-tween-universe-in-8-...,"[loft bed frame, desk top, pillow, quilt cover..."


In [124]:
img_to_desc_df['Image'] = img_to_desc_df['Image'].str.replace('/static/images/', '', regex=False)
img_to_desc_df['Image'] = img_to_desc_df['Image'].str.replace('.jpg', '', regex=False)
img_to_desc_df[['Description', 'Name']] = img_to_desc_df['Description'].apply(pd.Series)


In [125]:
img_to_desc_df['Image'] = img_to_desc_df['Image'].str.split('/').str[-1]


In [126]:
img_to_desc_df = img_to_desc_df.rename(columns={'Image': 'ProductID'})
img_to_desc_df.head()

,ProductID,Description,Name
0,500.210.76,Your child can use it for different purposes. ...,MLA
1,802.538.09,Turned upside down the bed quickly converts fr...,KURA
2,802.962.48,"Made from 100% cotton, a natural material that...",STICKAT
3,502.962.97,Clever storage solution that you can hang on o...,STICKAT
4,203.086.97,A playful and sturdy storage series for storin...,TROFAST


In [134]:
final_products_df = img_to_desc_df.merge(products_df, on=['ProductID', 'Description', 'Name'])



In [135]:
final_products_df.head()

,ProductID,Description,Name,Type,Color,Size,Image
0,500.210.76,Your child can use it for different purposes. ...,MLA,Easel,"Softwood,white",,images/objects/500.210.76.jpg
1,802.538.09,Turned upside down the bed quickly converts fr...,KURA,Reversible bed,"White,pine",90x200 cm,images/objects/802.538.09.jpg
2,802.538.09,Turned upside down the bed quickly converts fr...,KURA,Reversible bed,"White,pine",90x200 cm,images/objects/802.538.09.jpg
3,802.962.48,"Made from 100% cotton, a natural material that...",STICKAT,Quilt cover and pillowcase,"Turquoise,lilac","150x200,50x80 cm",images/objects/802.962.48.jpg
4,502.962.97,Clever storage solution that you can hang on o...,STICKAT,Bed pocket,Orange,39x30 cm,images/objects/502.962.97.jpg


In [144]:
# Assuming final_products_df is your DataFrame
blank_entries_count = final_products_df.applymap(lambda x: x == '').sum()

# Now, blank_entries_count contains the count of blank entries in each column
print(blank_entries_count)


ProductID      0
Description    0
Name           0
Type           0
Color          0
Size           0
Image          0
dtype: int64


In [143]:
import pandas as pd

# Assuming your DataFrame is named 'df'
columns_to_replace = ['Description', 'Color', 'Size']

# Replace empty strings with 'Not known' in specified columns
for column in columns_to_replace:
    final_products_df[column] = final_products_df[column].replace('', 'Not known')

In [145]:
final_products_df.to_csv('/Users/kabir/FRE-7773-Project/data/clean_data/products.csv')